In [1]:
import ee
import geemap
import geemap.colormaps as cm


In [2]:
Map = geemap.Map()


(google-chrome:104094): Gtk-WARNING **: 23:12:49.094: Theme parsing error: gtk.css:73:46: The style property GtkScrolledWindow:scrollbars-within-bevel is deprecated and shouldn't be used anymore. It will be removed in a future version


Opening in existing browser session.


[104148:104148:0100/000000.310321:ERROR:vaapi_wrapper.cc(1136)] vaQuerySurfaceAttributes failed, VA error: invalid parameter
[104148:104148:0100/000000.310365:ERROR:vaapi_wrapper.cc(1083)] FillProfileInfo_Locked failed for va_profile VAProfileH264Main and entrypoint VAEntrypointVLD
[104148:104148:0100/000000.310382:ERROR:vaapi_wrapper.cc(1136)] vaQuerySurfaceAttributes failed, VA error: invalid parameter
[104148:104148:0100/000000.310392:ERROR:vaapi_wrapper.cc(1083)] FillProfileInfo_Locked failed for va_profile VAProfileH264High and entrypoint VAEntrypointVLD


Enter verification code:  4/1AX4XfWi3iWaPPJJQ2vkiZC7cvc_BT87zlYw4Cn7pkeq1SO1ofQgweqJwRx4



Successfully saved authorization token.


In [3]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
# Map.add_legend(builtin_legend='ALOS_landforms')

In [5]:
fc = ee.FeatureCollection('users/giswqs/public/countries')
# Map.addLayer(fc, {}, 'Countries')

In [6]:
# rec = ee.Geometry.BBox(-179, -50, 179, 50)
# Map.addLayer(rec, {}, 'Rec')

In [7]:
landforms = ee.Image('CSP/ERGo/1_0/Global/ALOS_landforms').select('constant')
landformsVis = {
    'min': 11.0,
    'max': 42.0,
    'palette': [
        '141414',
        '383838',
        '808080',
        'EBEB8F',
        'F7D311',
        'AA0000',
        'D89382',
        'DDC9C9',
        'DCCDCE',
        '1C6330',
        '68AA63',
        'B5C98E',
        'E1F0E5',
        'a975ba',
        '6f198c',
    ],
}

In [8]:
Map.addLayer(landforms, landformsVis, 'Landforms')

In [9]:
srtm = ee.Image("CGIAR/SRTM90_V4")
hillsahde = ee.Terrain.hillshade(srtm)

In [10]:
# ned = ee.Image('USGS/3DEP/10m')
ned = ee.Image('USGS/SRTMGL1_003').select('elevation')

In [11]:
vis = {'min': 0, 'max': 6000, 'palette': cm.palettes.dem}

In [12]:
# Map.addLayer(hillsahde, {}, 'Hillshade')
Map.addLayer(srtm, vis, 'SRTM')

In [13]:
Map.addLayer(ned, vis, "NED")

In [14]:
srtm_blend = geemap.blend(top_layer=srtm, top_vis=vis)
Map.addLayer(srtm_blend, {}, 'SRTM Blend')

In [15]:
ned_blend = geemap.blend(top_layer=ned, top_vis=vis)
Map.addLayer(ned_blend, {}, 'NED Blend')

In [16]:
nlcd = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select('landcover')
nlcd_vis = {'bands': ['landcover']}
Map.addLayer(nlcd, nlcd_vis, 'NLCD')

In [17]:
diff = srtm.subtract(ned)

In [18]:
Map.addLayer(diff, {'min': -20, 'max': 20, 'palette': cm.palettes.coolwarm}, 'Diff')

In [19]:
# geemap.image_mean_value(diff, scale=500).getInfo()

In [20]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [21]:
hist = diff.reduceRegion(**{
    'reducer': ee.Reducer.histogram(maxBuckets=20),
    'bestEffort': True}).getInfo()

In [22]:
hist['elevation']['bucketMeans']

[-1565,
 -1457.6666666666667,
 -1334.4,
 -1206.4615384615386,
 -1075.7575757575758,
 -947.8666666666667,
 -819.8530612244898,
 -692.6904647910394,
 -568.6961611070544,
 -437.4882060344948,
 -308.06596373464555,
 -177.07148573224558,
 -18.59822132002834,
 15.869524548306048,
 175.7559097540676,
 306.8047520726253,
 437.56472404462795,
 566.0047005845051,
 692.2957829508376,
 818.9032417452576,
 944.2295081967213,
 1060.5384615384614,
 1190,
 1384,
 1472,
 1571.3333333333333]

In [23]:
hist['elevation']['histogram']

[1,
 3,
 5,
 13,
 33,
 90,
 245,
 622.5019607843137,
 1761.5372549019608,
 4500.141176470588,
 13525.792156862746,
 50807.560784313704,
 1157331.9372549045,
 1394831.2941176305,
 51963.278431372564,
 13069.054901960786,
 4252.321568627452,
 1469.9882352941174,
 572.0941176470589,
 182.5450980392157,
 61,
 26,
 14,
 2,
 0,
 3]

In [ ]:
labels = geemap.builtin_legends['ALOS_landforms'].keys()

In [ ]:
df = geemap.image_stats_by_zone(diff, landforms, reducer='mean', labels=labels)
df

In [ ]:
labels = [
    "Peak/ridge (warm)",
    "Peak/ridge",
    "Peak/ridge (cool)",
    "Mountain/divide",
    "Cliff",
    "Upper slope (warm)",
    "Upper slope",
    "Upper slope (cool)",
    "Upper slope (flat)",
    "Lower slope (warm)",
    "Lower slope",
    "Lower slope (cool)",
    "Lower slope (flat)",
    "Valley",
    "Valley (narrow)",
]

In [ ]:
df['label'] = labels

In [ ]:
df

In [ ]:
landform_fc = landforms.reduceToVectors(
    **{
        'geometry': Map.user_roi,
        'scale': 1000,
    }
)

In [ ]:
# Map.addLayer(landform_fc, {}, 'Landform Vector')

In [ ]:
result = diff.reduceRegions(
    **{'collection': landform_fc, 'reducer': ee.Reducer.mean(), 'scale': 100}
)

In [ ]:
c22 = diff.updateMask(landforms.eq(22)).selfMask()

In [ ]:
Map.addLayer(c22, {}, 'C22')

In [ ]:
from geemap.datasets import DATA

In [ ]:
DATA.users_giswqs_public_us_states

In [ ]:
fc = ee.FeatureCollection('users/giswqs/public/countries')

In [ ]:
# geemap.zonal_stats(c22, fc, 'stats_mean.csv', statistics_type='MEAN')

In [ ]:
geom = landforms.geometry().bounds()

In [ ]:
# values = landforms.reduceRegions(**{
#     'reducer': ee.Reducer.frequencyHistogram(),
#     'collection': ee.FeatureCollection([ee.Feature(geom)]),
#     'scale': 1000
# })

In [ ]:
# values.getInfo()

In [ ]:
values = geemap.image_value_list(landforms, region=geom, return_hist=False)

In [ ]:
values.getInfo()

In [ ]:
result = geemap.image_stats_by_zone(diff, landforms, reducer='mean')
result

In [ ]:
geemap.image_stats_by_zone(diff, nlcd, reducer='maximum')

In [ ]:
result.aggregate_array('zone').getInfo()

In [ ]:
result.aggregate_array('elevation').getInfo()

In [ ]:
result.toDictionary(['elevation']).getInfo()

In [ ]:
isinstance(ee.Reducer.histogram(), ee.Reducer)

In [ ]:
geom.getInfo()

In [ ]:
result = c22.reduceRegion(
    **{'reducer': ee.Reducer.mean(), 'scale': 1000, 'geometry': Map.user_roi}
)

In [ ]:
result.getInfo()

In [ ]:
ee.Image().set(result).set({'class': 22}).getInfo()